# Sprawozdanie z **Obliczeń ewolucyjnych**
## Zadanie 2 

# Konfiguracja
Przy pomocy klasy Config można wybrać zastosowany algorytm oraz określić jego parametry:    
- w - współczynnik zmęczenia.   
- c1 - współczynnik przyspieszenia w kierunku własnego minimum cząstki
- c2 - współczynnik przyspieszenia w kierunku globalnego minimum cząstek
- iterations - ilość iteracji algorytmu  
- target_error - najniższy stopień błędu - po osiągnięciu tego wyniku algorytm zatrzymuje się  
- n_particles - ilość cząstek  
- arguments_dimensions - liczba wymiarów, w których rozkładane są cząsteczki   
- d_min - minimalna wartość współrzędnej dla każdego z wymiarów   
- d_max - maksymalna wartość współrzędnej dla każdego z wymiarów   
- fitness - wybór funkcji dostosowującej położenie cząstek         
 Dla celów badawczych, w dalszej części zaprezentowane są różne zestawy wartości konfiguracyjnych.

In [1]:
!pip install deap k3d > /dev/null
%matplotlib inline
from pathlib import Path
import json

class Config(object):
    def __init__(self, algorithmType, w, c1, c2, custom_c2, iterations, target_error, n_particles, arguments_dimensions, d_min, d_max, fitness):
        self.algorithmType = algorithmType
        self.w = w
        self.c1 = c1
        self.c2 = c2
        self.custom_c2 = custom_c2
        self.iterations = iterations
        self.target_error = target_error
        self.n_particles = n_particles
        self.arguments_dimensions = arguments_dimensions
        self.d_min = d_min
        self.d_max = d_max
        self.d_max = d_max
        self.fitness = fitness

def as_config(dct):
    return Config(
        dct['algorithmType'],
        dct['w'],
        dct['c1'],
        dct['c2'],
        dct['custom_c2'] if dct.get('custom_c2') else False,
        dct['iterations'], 
        dct['target_error'], 
        dct['n_particles'],
        dct['arguments_dimensions'],
        dct['d_min'], 
        dct['d_max'],
        dct['fitness']
        )

# Algorytm PSO

### Imports

In [2]:
#deap
import operator
import random

import numpy
import math

from deap import base
from deap import benchmarks
from deap import creator
from deap import tools

#random
from numpy import random as nrand
nrand.seed(0)
from random import seed
seed(0)

### Creator

In [3]:
creator.create("FitnessMax", base.Fitness, weights=(-1.0,))
creator.create("Particle", list, fitness=creator.FitnessMax, speed=list, 
    smin=None, smax=None, best=None)

def generate(size, pmin, pmax, smin, smax):
    part = creator.Particle(random.uniform(pmin, pmax) for _ in range(size)) 
    part.speed = [random.uniform(smin, smax) for _ in range(size)]
    part.smin = smin
    part.smax = smax
    return part

### Update particles

In [4]:
def updateParticle(part, best, phi1, phi2):
    u1 = (random.uniform(0, phi1) for _ in range(len(part)))
    u2 = (random.uniform(0, phi2) for _ in range(len(part)))
    v_u1 = map(operator.mul, u1, map(operator.sub, part.best, part))
    v_u2 = map(operator.mul, u2, map(operator.sub, best, part))
    part.speed = list(map(operator.add, part.speed, map(operator.add, v_u1, v_u2)))
    for i, speed in enumerate(part.speed):
        if abs(speed) < part.smin:
            part.speed[i] = math.copysign(part.smin, speed)
        elif abs(speed) > part.smax:
            part.speed[i] = math.copysign(part.smax, speed)
    part[:] = list(map(operator.add, part, part.speed))

### Toolbox

In [5]:
toolbox = base.Toolbox()
toolbox.register("particle", generate, size=2, pmin=-6, pmax=6, smin=-3, smax=3)
toolbox.register("population", tools.initRepeat, list, toolbox.particle)
toolbox.register("update", updateParticle, phi1=2.0, phi2=2.0)
toolbox.register("evaluate", benchmarks.h1)

### App

In [6]:
class App():
    def run(self):
        pop = toolbox.population(n=5)
        stats = tools.Statistics(lambda ind: ind.fitness.values)
        stats.register("avg", numpy.mean)
        stats.register("std", numpy.std)
        stats.register("min", numpy.min)
        stats.register("max", numpy.max)

        logbook = tools.Logbook()
        logbook.header = ["gen", "evals"] + stats.fields

        GEN = 1000
        best = None

        for g in range(GEN):
            for part in pop:
                part.fitness.values = toolbox.evaluate(part)
                if not part.best or part.best.fitness < part.fitness:
                    part.best = creator.Particle(part)
                    part.best.fitness.values = part.fitness.values
                if not best or best.fitness < part.fitness:
                    best = creator.Particle(part)
                    best.fitness.values = part.fitness.values
            for part in pop:
                toolbox.update(part, best)

            # Gather all the fitnesses in one list and print the stats
            logbook.record(gen=g, evals=len(pop), **stats.compile(pop))
            print(logbook.stream)

        return pop, logbook, best

# Badania

In [7]:
# json_config ="""
# {
#     "algorithmType": "pso",
#     "fitness": "sincos",
#     "w": 0.5,
#     "c1": 0.8,
#     "c2": 0.9,
#     "custom_c2": true,
#     "target_error": 0,
#     "iterations": 400,
#     "n_particles": 1000,
#     "arguments_dimensions": 2,
#     "d_min": -10,
#     "d_max": 10
# }"""
# cfg = json.loads(json_config, object_hook = as_config)
App().run()

gen	evals	avg      	std      	min      	max     
0  	5    	0.0776992	0.0458332	0.0132121	0.156738
1  	5    	0.143405 	0.0672199	0.029804 	0.238798
2  	5    	0.0855771	0.0328971	0.0466571	0.131825
3  	5    	0.0924496	0.0259555	0.0519143	0.125961
4  	5    	0.046369 	0.0140306	0.0284203	0.0689271
5  	5    	0.108506 	0.0312518	0.0732267	0.149836 
6  	5    	0.0543751	0.0252378	0.0113183	0.0795382
7  	5    	0.0500722	0.0387547	0.0142216	0.120008 
8  	5    	0.0573202	0.0353404	0.0084253	0.116879 
9  	5    	0.0663867	0.0538478	0.0060151	0.167364 
10 	5    	0.0752277	0.0615815	0.0102292	0.168221 
11 	5    	0.0756549	0.0557321	0.00548955	0.155304 
12 	5    	0.0686154	0.0433866	0.000812849	0.124741 
13 	5    	0.0557506	0.0396899	0.0169189  	0.124461 
14 	5    	0.0828772	0.0218348	0.057341   	0.111699 
15 	5    	0.0744713	0.0154072	0.0598919  	0.0984856
16 	5    	0.0698719	0.0371708	0.0110435  	0.112847 
17 	5    	0.102579 	0.0387692	0.0382408  	0.158739 
18 	5    	0.0668744	0.0471575	0.00214721 	

691	5    	0.047289 	0.0312413 	0.00669316 	0.0829769
692	5    	0.080188 	0.0149075 	0.0626908  	0.105225 
693	5    	0.0498351	0.0347564 	0.00852105 	0.107594 
694	5    	0.0892569	0.0247658 	0.0560902  	0.112118 
695	5    	0.0728234	0.0463167 	0.0140188  	0.134342 
696	5    	0.069078 	0.0274065 	0.0220409  	0.105073 
697	5    	0.0739057	0.0338357 	0.021999   	0.120813 
698	5    	0.0767328	0.0375989 	0.0257552  	0.129016 
699	5    	0.0787195	0.0342362 	0.0359462  	0.129155 
700	5    	0.0769517	0.0238082 	0.0451312  	0.0985057
701	5    	0.0547332	0.0286548 	0.0130981  	0.087135 
702	5    	0.0602788	0.0223611 	0.0253975  	0.0836125
703	5    	0.0594762	0.0348841 	0.0141748  	0.104257 
704	5    	0.0983372	0.0267943 	0.0710253  	0.133519 
705	5    	0.0648414	0.0303558 	0.0218643  	0.101724 
706	5    	0.0888354	0.0280998 	0.0340064  	0.114356 
707	5    	0.0747691	0.0269327 	0.0249475  	0.101793 
708	5    	0.0494081	0.0222644 	0.0247137  	0.0787525
709	5    	0.0716259	0.0346092 	0.0284382  	0.1

([[-2.9944997263724815, 3.433309586491963],
  [-1.5559504431949307, 3.6379429139581543],
  [1.7360968619695853, 4.986254472033032],
  [-0.9194293727297724, 6.533970246319695],
  [-3.741326331835178, 3.9564876993916642]],
 [{'gen': 0,
   'evals': 5,
   'avg': 0.07769923001726461,
   'std': 0.045833163536919234,
   'min': 0.013212126823776373,
   'max': 0.1567375897637734},
  {'gen': 1,
   'evals': 5,
   'avg': 0.14340478912140772,
   'std': 0.06721987963349084,
   'min': 0.029803956960898305,
   'max': 0.23879765152405785},
  {'gen': 2,
   'evals': 5,
   'avg': 0.08557712643228736,
   'std': 0.03289714964245554,
   'min': 0.04665707995120681,
   'max': 0.13182462739980294},
  {'gen': 3,
   'evals': 5,
   'avg': 0.09244955454164412,
   'std': 0.025955536370519713,
   'min': 0.051914336205459974,
   'max': 0.12596062828758894},
  {'gen': 4,
   'evals': 5,
   'avg': 0.04636900543548597,
   'std': 0.014030623236729517,
   'min': 0.02842026377611954,
   'max': 0.06892706326211502},
  {'gen':